# Consulta de  ejecuciones de jobs en Dataproc

Pasos a realizar
----------------

1) Activar conexión VPN remota (Pulse), si es necesario

2) Descargar jobs desde 'jobs_dataproc':  
https://docs.google.com/spreadsheets/d/1j-ndJrhlfHb70O730pUoYSWOZrtphbJ9Z4qy-BIVSis/edit#gid=639609316  
Guardar en `./jobs/jobs.csv`  
El csv, como mínimo debe tener las siguientes columnas:  
    ```
    uuaa,job
    B0HN,b0hn-gl-spk-qlt-inmXXXXXXXX-NN
    ```

3) Generar token en: https://portunus.live-01.nextgen.igrupobbva/v1/requestToken.html
- Razon:  
Consulta de ejecuciones de Jobs de dataproc en live mediante peticiones get
- Nivel **Alto**  
Conceder acceso y copiar clipboard.  
Guardar en `./token.txt`

4) Ejecutar script (restart kernell and run all cells)  
Resultado de la consulta en `./runs/runs_{fecha}.csv`

Libraries

In [1]:
import pandas as pd
import requests
import json
import warnings
from tqdm import tqdm
from datetime import date
import time
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

API_path    = "https://dataproc.live-01.nextgen.igrupobbva/v3"
token       = open("token.txt", "r").read()
header_api  = {"Authorization":"Bearer {t}".format(t=token)}

Functions

In [2]:
def get_ns(country_ns  = "glo", type_ns = "pro", header_api = header_api):
    """
    Returns and writes Dataframe with Namespace asociated to all UUAAs
    
    Parameters
    ----------
    country_ns: filter for country, default "glo"
    type_ns: filter for type of ns, default "pro"
    header_api: Header of request for Dataproc Api

    Returns
    -------
    data_runs: list with with existing job runs data
    no_runs: list with with no existing job runs data
    """
    url = API_path + "/ns-configs?pageSize=50"
    r_ns = requests.get(url, headers=header_api, params={'offset':0}, verify=False)
    pages = json.loads(r_ns.text)['pagination']['totalPages']

    data = []
    for p in tqdm(range(pages)):
        url = API_path + "/ns-configs?pageSize=50&paginationKey=" + str(p)
        r_ns = requests.get(url, headers=header_api, params={'offset':0}, verify=False)
        if r_ns.ok:
            time.sleep(0.25)
            data.append(pd.json_normalize(json.loads(r_ns.text)['data']).query('enabled==True')[["_id","params.uuaa","params.country"]])
        elif r_ns.status_code == 403:
            print("Error 403, vuelve a pedir el token con nivel alto")
        else:
            print("Error: ",r_ns.status_code)
    df_ns = pd.concat(data,axis=0).reset_index(drop=True)
    df_ns.columns = ['id_ns','uuaa','country']
    df_ns['type'] = df_ns.id_ns.str[-3:]
    print("ns type:")
    print(df_ns['type'].value_counts())
    print("ns country:")
    print(df_ns['country'].value_counts())
    print("filtramos country:glo y type:pro")
    df_ns.query('country==@country_ns', inplace=True)
    df_ns.query('type==@type_ns', inplace=True)
    df_ns.to_csv('./ns/ns.csv',sep=';',index=False)
    return df_ns

def get_runs_by_job(df_jobs_ns, header_api = header_api):
    """
    Returns lists with job runs and no job runs data

    Parameters
    ----------
    df_jobs_ns: DataFrame with "id_ns" (Dataproc NameSpace of job) and "job" (Dataproc Name of job) columns.
    header_api: Header of request for Dataproc Api

    Returns
    -------
    data_runs: list with with existing job runs data
    no_runs: list with with no existing job runs data
    """
    data_runs = []
    no_runs = []
    print('Total {0} jobs'.format(str(len(df_jobs_ns))))
    for index, row in tqdm(df_jobs_ns.iterrows()):
        url = API_path + "/ns/{n}/jobs/{j}/runs".format(n=row['id_ns'], j=row['job'])
        #print(url)
        r_run = requests.get(url, headers=header_api, verify=False)
        if r_run.ok and json.loads(r_run.text)['data']:
            if "CLOSING_DATE" in json.loads(r_run.text)['data'][0]['runEnv']:
                runs = pd.json_normalize(json.loads(r_run.text)['data'])[['_id','status.id','runEnv.CLOSING_DATE','jobParams.configUrl','_createdAt','status.updatedAt','_createdBy']]
            elif "DATIO_DATE" in json.loads(r_run.text)['data'][0]['runEnv']:
                runs = pd.json_normalize(json.loads(r_run.text)['data'])[['_id','status.id','runEnv.DATIO_DATE','jobParams.configUrl','_createdAt','status.updatedAt','_createdBy']]
            elif "CUTOFF_DATE" not in json.loads(r_run.text)['data'][0]['runEnv']:
                print('INCLUIR PARÁMETRO CUT_OFF_DATE ALTERNATIVO')
                print(json.loads(r_run.text)['data'])
                # runs = pd.json_normalize(json.loads(r_run.text)['data'])[['_id','status.id','runEnv.CUTOFF_DATE','jobParams.configUrl','_createdAt','status.updatedAt','_createdBy']]
            else:
                runs = pd.json_normalize(json.loads(r_run.text)['data'])[['_id','status.id','runEnv.CUTOFF_DATE','jobParams.configUrl','_createdAt','status.updatedAt','_createdBy']]
            runs.columns = ['run_id','status','CUTOFF_DATE','configUrl','run_createdAt','status_updatedAt','run_createdBy']
            runs['job'] = row['job']
            runs['id_ns'] = row['id_ns']
            data_runs.append(runs)
        elif (r_run.status_code == 403) or (r_run.status_code == 401):
            print("Error 403 o 401, vuelve a pedir el token con nivel alto: ", r_run.status_code)
        elif r_run.status_code == 200:
            print("-NA (job exists, no runs):",r_run.status_code, url, row['job'])
            no_runs.append([row['id_ns'], row['job'], r_run.status_code, json.loads(r_run.text)])
        elif r_run.status_code == 502:
            print("-NA (COMPROBAR en DP, volver a lanzar API):",r_run.status_code, url, row['job'])
            no_runs.append([row['id_ns'], row['job'], r_run.status_code, json.loads(r_run.text)])
        else:
            print("-NA :",r_run.status_code, url)
            no_runs.append([row['id_ns'], row['job'], r_run.status_code, json.loads(r_run.text)])
        time.sleep(2)
    return data_runs, no_runs

REQUEST: All ns's

In [3]:
# df_ns = get_ns()
df_ns = pd.read_csv('./ns/ns.csv', sep=';')
df_ns.tail(2)

,id_ns,uuaa,country,type
554,comh-pro,comh,glo,pro
555,kple-pro,kple,glo,pro


Read jobs & merge ns's

In [4]:
df_jobs = pd.read_csv('./jobs/jobs.csv').dropna(subset='job',axis=0)
df_jobs.uuaa = df_jobs.uuaa.str.strip().str.lower()
df_jobs.job = df_jobs.job.str.strip()

df_jobs_ns = df_jobs.merge(df_ns, on='uuaa', how='left', indicator=True).query('_merge=="both"').drop(columns='_merge')
print('UUAAs sin ns:', df_jobs.merge(df_ns, on='uuaa', how='left', indicator=True).query('_merge=="left_only"').uuaa.unique())
df_jobs_ns.tail(2)

UUAAs sin ns: ['jobs']


,uuaa,job,conf,id_ns,country,type
222,g5m3,g5m3-gl-hmm-qlt-inmmrulesustmboprtincibsust-01,G5M3_sust-mob-operations-cib_G5M3_sust-mob-ope...,gl.g5m3.app-id-23164.pro,glo,pro
223,rzua,rzua-gl-hmm-qlt-inmmrulecntrctoukcog-01,RZUA_contract-output-emissions_KCOG_country_MM...,gl.rzua.app-id-19546.pro,glo,pro


REQUEST: Runs by job 

In [5]:
data_runs, no_runs = get_runs_by_job(df_jobs_ns)
df_runs     = pd.concat(data_runs, axis=0).reset_index(drop=True)
df_noruns   = pd.DataFrame(no_runs,columns=['id_ns','job','status_code','content'])
df_runs.tail(2)

Total 223 jobs


130it [05:21,  2.26s/it]

-NA (job exists, no runs): 200 https://dataproc.live-01.nextgen.igrupobbva/v3/ns/gl.rzua.app-id-19546.pro/jobs/rzua-gl-hmm-qlt-inmmmrsustopratinsciboprtci-01/runs rzua-gl-hmm-qlt-inmmmrsustopratinsciboprtci-01


188it [07:31,  2.29s/it]

-NA (job exists, no runs): 200 https://dataproc.live-01.nextgen.igrupobbva/v3/ns/gl.l9ct.app-id-219787.pro/jobs/l9ct-gl-hmm-qlt-inmmvpesgfactreply-01/runs l9ct-gl-hmm-qlt-inmmvpesgfactreply-01


...

-NA (job exists, no runs): 200 https://dataproc.live-01.nextgen.igrupobbva/v3/ns/gl.g5m3.app-id-23164.pro/jobs/g5m3-gl-hmm-qlt-inmmrulesustmboprtincibsust-01/runs g5m3-gl-hmm-qlt-inmmrulesustmboprtincibsust-01


222it [08:46,  2.20s/it]

-NA (job exists, no runs): 200 https://dataproc.live-01.nextgen.igrupobbva/v3/ns/gl.rzua.app-id-19546.pro/jobs/rzua-gl-hmm-qlt-inmmrulecntrctoukcog-01/runs rzua-gl-hmm-qlt-inmmrulecntrctoukcog-01


223it [08:48,  2.37s/it]


,run_id,status,CUTOFF_DATE,configUrl,run_createdAt,status_updatedAt,run_createdBy,job,id_ns
1942,801f9898-4ac6-4bb1-bafe-1d9c302b9788,FAILURE,2024-08-19,https://artifactory-gdt.central-01.nextgen.igr...,2024-08-19T11:45:24.842Z,2024-08-19T12:25:44.355Z,SENTRY-BOT@DATAPROC.ECS,kfcm-gl-hmm-qlt-inmmvpfeedbackoptions-01,gl.kfcm.app-id-26255.pro
1943,2083940f-1340-4351-8b8c-1e208227127f,FAILURE,2024-07-31,https://artifactory-gdt.central-01.nextgen.igr...,2024-08-19T14:17:45.977Z,2024-08-19T14:24:44.705Z,SENTRY-BOT@DATAPROC.ECS,l9ct-gl-hmm-qlt-inmmrulegpcustextksag-01,gl.l9ct.app-id-219787.pro


DF FINAL

In [6]:
dir_df_final = 'runs/runs_{fecha}.csv'.format(fecha = date.today().strftime('%Y_%m_%d'))

df = df_jobs.merge(df_runs, on='job', how='left')
df.run_createdAt            = pd.to_datetime(df.run_createdAt)#, format = "mixed")
df.status_updatedAt         = pd.to_datetime(df.status_updatedAt)#, format = "mixed")
df["status_updatedAt"]      = df.status_updatedAt.dt.date
df["fecha_actualizacion"]   = date.today()
df = df.sort_values(['job','status_updatedAt','status'], ascending=False)
df.to_csv(dir_df_final, index = False); print("Guardado df final en:\n", dir_df_final, sep='')
df.head(2)

Guardado df final en:
runs/runs_2024_10_15.csv


,uuaa,job,conf,run_id,status,CUTOFF_DATE,configUrl,run_createdAt,status_updatedAt,run_createdBy,id_ns,fecha_actualizacion
1780,scat,scat-gl-qlt-inmmvpsustmaster-01,SCAT_sustainability-master_MRField.conf,fd7253fa-54c1-4825-ae93-0e96f8ee284e,SUCCESS,2024-10-14,https://artifactory-gdt.central-01.nextgen.igr...,2024-10-14 23:33:57.531000+00:00,2024-10-14,SENTRY-BOT@DATAPROC.ECS,gl.scat.app-id-1302468.pro,2024-10-15
1781,scat,scat-gl-qlt-inmmvpsustmaster-01,SCAT_sustainability-master_MRField.conf,dded6c5d-0850-4cf3-9613-c6a199b8ea95,SUCCESS,2024-10-13,https://artifactory-gdt.central-01.nextgen.igr...,2024-10-13 23:31:08.078000+00:00,2024-10-13,SENTRY-BOT@DATAPROC.ECS,gl.scat.app-id-1302468.pro,2024-10-15


In [7]:
dir_df_nnoruns = 'runs/no_runs/no_runs_{fecha}.csv'.format(fecha = date.today().strftime('%Y_%m_%d'))
df_noruns.to_csv(dir_df_nnoruns, index = False); print("Guardado df no runs en:\n", dir_df_nnoruns, sep='')

df_noruns

Guardado df no runs en:
runs/no_runs/no_runs_2024_10_15.csv


,id_ns,job,status_code,content
0,gl.rzua.app-id-19546.pro,rzua-gl-hmm-qlt-inmmmrsustopratinsciboprtci-01,200,"{'data': [], 'pagination': {'links': {'first':..."
1,gl.l9ct.app-id-219787.pro,l9ct-gl-hmm-qlt-inmmvpesgfactreply-01,200,"{'data': [], 'pagination': {'links': {'first':..."
2,gl.l9ct.app-id-219787.pro,l9ct-gl-hmm-qlt-inmstresgfactreply-01,200,"{'data': [], 'pagination': {'links': {'first':..."
3,gl.l9ct.app-id-219787.pro,l9ct-gl-hmm-qlt-inmmvpesgquestionscore-01,200,"{'data': [], 'pagination': {'links': {'first':..."
4,gl.l9ct.app-id-219787.pro,l9ct-gl-hmm-qlt-inmstresgquestionscore-01,200,"{'data': [], 'pagination': {'links': {'first':..."
5,gl.kfcm.app-id-26255.pro,kfcm-gl-hmm-qlt-inmstrfdbkfilldfrmfld-01,200,"{'data': [], 'pagination': {'links': {'first':..."
6,gl.kfcm.app-id-26255.pro,kfcm-gl-hmm-qlt-inmstrsaturationrules-01,200,"{'data': [], 'pagination': {'links': {'first':..."
7,gl.kfcm.app-id-26255.pro,kfcm-gl-hmm-qlt-inmstrfeedbckfrmfield-01,200,"{'data': [], 'pagination': {'links': {'first':..."
8,gl.kfcm.app-id-26255.pro,kfcm-gl-hmm-qlt-inmmvpfeedbckfrmshown-01,200,"{'data': [], 'pagination': {'links': {'first':..."
9,gl.kfcm.app-id-26255.pro,kfcm-gl-hmm-qlt-inmstrfeedbackforms-01,200,"{'data': [], 'pagination': {'links': {'first':..."


In [8]:
df_noruns.job.tolist()

['rzua-gl-hmm-qlt-inmmmrsustopratinsciboprtci-01',
 'l9ct-gl-hmm-qlt-inmmvpesgfactreply-01',
 'l9ct-gl-hmm-qlt-inmstresgfactreply-01',
 'l9ct-gl-hmm-qlt-inmmvpesgquestionscore-01',
 'l9ct-gl-hmm-qlt-inmstresgquestionscore-01',
 'kfcm-gl-hmm-qlt-inmstrfdbkfilldfrmfld-01',
 'kfcm-gl-hmm-qlt-inmstrsaturationrules-01',
 'kfcm-gl-hmm-qlt-inmstrfeedbckfrmfield-01',
 'kfcm-gl-hmm-qlt-inmmvpfeedbckfrmshown-01',
 'kfcm-gl-hmm-qlt-inmstrfeedbackforms-01',
 'kfcm-gl-hmm-qlt-inmstrfeedbackoptions-01',
 'l9ct-gl-hmm-qlt-inmmvpesgextdcoanloilgas-01',
 'l9ct-gl-hmm-qlt-inmstresgextdcoanloilgas-01',
 'l9ct-gl-hmm-qlt-inmmvpprodassetcorelations-01',
 'l9ct-gl-hmm-qlt-inmstrprodassetcorelations-01',
 'l9ct-gl-hmm-qlt-inmmvpprodassetsoilgasann-01',
 'l9ct-gl-hmm-qlt-inmstrprodassetsoilgasann-01',
 'l9ct-gl-hmm-qlt-inmmvpprodassetsoilgas-01',
 'l9ct-gl-hmm-qlt-inmstrprodassetsoilgas-01',
 'l9ct-gl-hmm-qlt-inmmvpproductionassets-01',
 'l9ct-gl-hmm-qlt-inmstrproductionassets-01',
 'l9ct-gl-hmm-qlt-inmmvppro